<a href="https://colab.research.google.com/github/Shashiawari/Sentimental_Analysis/blob/main/train_model_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Read the dataset csv file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/
! ls

/content/drive/MyDrive
 030413.hill.think.and.grow.rich.pdf			     results
'Adam Oh - Trapped In My Mind ( ).mp3'			     Screenshot_20230620_162010_Chrome.jpg
'ai m [Autosaved] (1).pptx'				    'Selena Gomez, Camilo - 999.mp3'
'ai m.gslides'						     sentimental
'ai m.pptx'						     sentitrans.csv
 certifi						    'Shawn Mendes - Treat You Better.mp3'
'Colab Notebooks'					     spam_model
 Contacts.vcf						     train_model.ipynb
 dataset.csv						    'Untitled form (1).gform'
'Dharia - (Uh Nai Na) Sugar And Brownies.mp3'		    'Untitled form (2).gform'
 FINAL_OPTIONS.gdoc					    'Untitled form (3).gform'
 FINAL_OPTIONS.pdf					    'Untitled form (4).gform'
 GDToT							    'Untitled form (5).gform'
'HTML _ CSS_HTML _ CSS: Course Completion Certificate.pdf'  'Untitled form (6).gform'
'HVME - GOOSEBUMPS (Official Video).mp3'		    'Untitled form (7).gform'
'Kina - Can We Kiss Forever ft. Adriana Proenza.mp3'	    'Untitled form (8).gform'
'Kygo, Selena Gomez - It Ain t Me (Official Vid

In [ ]:
!pip install transformers[torch] pandas datasets pyarrow scikit-learn



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df = pd.read_csv('sentitrans.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Process the data

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def process_data(row):

    text = row['review']
    text = str(text)
    text = ' '.join(text.split())

    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)

    label = 0
    if row['sentiment'] == 'positive':
        label = 1

    encodings['label'] = label
    encodings['text'] = text

    return encodings

In [ ]:
processed_data = []

for i in range(len(df[:1000])):
    processed_data.append(process_data(df.iloc[i]))

## Generate the dataset

In [ ]:
from sklearn.model_selection import train_test_split

new_df = pd.DataFrame(processed_data)

train_df, valid_df = train_test_split(
    new_df,
    test_size=0.3,
    random_state=2022
)

In [ ]:
import pyarrow as pa
from datasets import Dataset

train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

## Create a model

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="./result", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer=tokenizer
)

## Train and Evaluate the model

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,1.134159
2,No log,0.628761
3,No log,0.665926


TrainOutput(global_step=300, training_loss=0.38175572713216144, metrics={'train_runtime': 66.1315, 'train_samples_per_second': 36.291, 'train_steps_per_second': 4.536, 'total_flos': 157866633216000.0, 'train_loss': 0.38175572713216144, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6659256219863892,
 'eval_runtime': 1.3904,
 'eval_samples_per_second': 143.839,
 'eval_steps_per_second': 17.98,
 'epoch': 3.0}

## Save the model

In [ ]:
model.save_pretrained('./model/')

## Load the model

In [ ]:
pip install Cmake

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_model = AutoModelForSequenceClassification.from_pretrained('./model/').to(device)

In [ ]:
from transformers import AutoTokenizer

new_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

## Get predictions

In [ ]:
import torch
import numpy as np

def get_prediction(text):
    encoding = new_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

    outputs = new_model(**encoding)

    logits = outputs.logits
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    sigmoid = torch.nn.Sigmoid()
    print(sigmoid)
    probs = sigmoid(logits.squeeze().cpu())
    probs = probs.detach().numpy()
    label = np.argmax(probs, axis=-1)

    if label == 1:
        return {
            'sentiment': 'Positive',
            'probability': probs[1]
        }
    else:
        return {
            'sentiment': 'Negative',
            'probability': probs[0]
        }

In [ ]:
get_prediction('it is nice')

Sigmoid()


{'sentiment': 'Positive', 'probability': 0.9127835}